<a href="https://colab.research.google.com/github/zideric/colab/blob/main/LSTM_e_GRU_con_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM e GRU con Keras
le reti long short-term memory sono delle reti ricorrenti che risolvono il problema della scomparsa del gradiente tra le diverse esecuzioni della rete.

le LSTM in sostanza aggiungono un canale prioritario alla rete, chiamato cell state o memory cell, che viaggia in parallelo al segnale della rete e immagazzina le informazioni sequenziali

Qui la [ricerca](https://www.bioinf.jku.at/publications/older/2604.pdf)

usiamo LSTM per la sentiment analysis dell'IMDB

In [1]:
from time import time

import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

## Scarichiamo il dataset
come al solito con Keras

In [2]:
from keras.datasets import imdb

num_words = 10000

(X_train, y_train),(X_test,y_test) = imdb.load_data(num_words = num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


## Processiamo i dati
usiamo sempre pad_sequences di keras per limitare le sequenze a 500 elementi (in questo caso per limitare le frasi a 500 parole). se è piu corta verranno aggiunti degli 0 finali

In [3]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

X_train.shape

(25000, 500)

## Creiamo il modello LSTM
il modelllo sarà cosi strutturato:
* primo strato esegue embedding creando 50 embedding vectors per ognuna delle 10000 parole del nostro dizionario
* il secondo è lo strato ricorrente di tipo LSTM
* il terzo strato calcolerà l'output della rete, trattandosi di classificazione bianria (postivia/negativa) la funzione sarà la sigmoide

In [4]:
from keras.layers import Embedding, LSTM

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          500000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                10624     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


compiliamo il modello ed eseguiamolo per 5 epoche

In [6]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train,batch_size=512, validation_split=0.2,epochs=5)
model.evaluate(X_test,y_test)

Epoch 1/5
40/40 [==============================] - 49s 1s/step - loss: 0.6549 - accuracy: 0.6209 - val_loss: 0.4609 - val_accuracy: 0.8096
Epoch 2/5
40/40 [==============================] - 46s 1s/step - loss: 0.4031 - accuracy: 0.8431 - val_loss: 0.6534 - val_accuracy: 0.7142
Epoch 3/5
40/40 [==============================] - 46s 1s/step - loss: 0.3347 - accuracy: 0.8696 - val_loss: 0.6682 - val_accuracy: 0.7660
Epoch 4/5
40/40 [==============================] - 46s 1s/step - loss: 0.3114 - accuracy: 0.8842 - val_loss: 0.3285 - val_accuracy: 0.8638
Epoch 5/5
782/782 [==============================] - 31s 40ms/step - loss: 0.3022 - accuracy: 0.8743


[0.3021528422832489, 0.8743199706077576]

il risultato è nettamente migliore di un RNN, c'è un po di overfitting che proviamo a mitigare utilizzando il Dropout

## Dropout di una RNN

in una RNN possiamo utilizzare due approcci per il dropout (2 tipologie):
* Dropout sull'input/output dello strato, esattamente come abbiamo già fatto con le altre architetture di reti neurali
* Dropout tra le esecuzion ricorrenti della rete, questo permette di ridurre l'overfitting nelle features che contengono le informazioni sulla seuqenza.

Per utilizzre il dropout sull'input di uno strato ricorrente, piuttosto che usare la classe Dropoiut, è consigliato sfruttare il parametro dropout delle classi SimpleRNN e LSTM. Per utilizzre il dropout ricorrente possiamo invece utilizzre il parametro recurrent_dropout delle classi SIMpleRNN e LSTM.

In [7]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.2))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


compiliamo e addestriamo per 5 epoche

In [8]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Epoch 1/5
40/40 [==============================] - 92s 2s/step - loss: 0.6793 - accuracy: 0.5853 - val_loss: 0.6905 - val_accuracy: 0.5748
Epoch 2/5
40/40 [==============================] - 89s 2s/step - loss: 0.4947 - accuracy: 0.7937 - val_loss: 0.5358 - val_accuracy: 0.7568
Epoch 3/5
40/40 [==============================] - 89s 2s/step - loss: 0.3685 - accuracy: 0.8578 - val_loss: 0.4262 - val_accuracy: 0.8356
Epoch 4/5
40/40 [==============================] - 88s 2s/step - loss: 0.3116 - accuracy: 0.8901 - val_loss: 0.3282 - val_accuracy: 0.8722
Epoch 5/5
782/782 [==============================] - 43s 55ms/step - loss: 0.4797 - accuracy: 0.8058


[0.47971367835998535, 0.8058000206947327]

l'overfitting è stato ridotto, proviamo a migliorare il modello aggiungendo altri strati ricrrenti alla rete.

## Aggiungiamo altri strati LSTM

possiamo aggingere altri strati ricorrenti nella solita maniera. di default la classe LSTM esegue il flattening della sequenza per poterla dare come input ad uno strato denso, possiamo modificare tale comportamento impostando il parametro return_sequences = True.

In [9]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 100)) #prima abbiamo eseguito con 50 embedding vectors
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 32)          17024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,025,377
Trainable params: 1,025,377
Non-trainable params: 0
_________________________________________________________________


compiliamo ed eseguiamo le 5 epoche. questa volta cronometriamo perche confronteremo questo risultato con la variante GRU

In [10]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
start_at=time()
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
print("Addestramento completato in %.f secondi (5 epoche)" % ((time()-start_at)))
model.evaluate(X_test, y_test)

Epoch 1/5
40/40 [==============================] - 203s 5s/step - loss: 0.6493 - accuracy: 0.6022 - val_loss: 0.3836 - val_accuracy: 0.8416
Epoch 2/5
40/40 [==============================] - 201s 5s/step - loss: 0.4084 - accuracy: 0.8226 - val_loss: 0.3211 - val_accuracy: 0.8644
Epoch 3/5
40/40 [==============================] - 202s 5s/step - loss: 0.3044 - accuracy: 0.8811 - val_loss: 0.3561 - val_accuracy: 0.8712
Epoch 4/5
40/40 [==============================] - 202s 5s/step - loss: 0.2961 - accuracy: 0.8871 - val_loss: 0.4549 - val_accuracy: 0.8558
Epoch 5/5
40/40 [==============================] - 202s 5s/step - loss: 0.2710 - accuracy: 0.9022 - val_loss: 0.3174 - val_accuracy: 0.8892
Addestramento completato in 1010 secondi (5 epoche)
782/782 [==============================] - 84s 107ms/step - loss: 0.3529 - accuracy: 0.8752


[0.3529222011566162, 0.8751999735832214]

## Grated Recurrent UNIT (GRU)
sono una tipologia di reti neurali ricorrenti che prendono spunto da e semplificano le LSTM. a differenza di quest'ultime le GRU richiedono meno calcoli tensoriali e quindi solitamente portano a risultati simili in minor tempo.
Possiamo aggiungere gli strati GRU con Keras tramite la classe GRU.

In [13]:
from keras.layers import Embedding, GRU, Dropout

model = Sequential()

model.add(Embedding(num_words, 100))
model.add(GRU(32, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
model.add(GRU(32, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
gru (GRU)                    (None, None, 32)          12864     
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6336      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,019,233
Trainable params: 1,019,233
Non-trainable params: 0
_________________________________________________________________


compiliamo e avviamo l'addestramento su 5 epoche cronometrando.

In [14]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
start_at=time()
model.fit(X_train,y_train,batch_size=512,validation_split=0.2,epochs=5)
print("Addestramento completato in %.f secondi (5 epoche)" % ((time()-start_at)))
model.evaluate(X_test, y_test)

Epoch 1/5
40/40 [==============================] - 168s 4s/step - loss: 0.6721 - accuracy: 0.5699 - val_loss: 0.5175 - val_accuracy: 0.7428
Epoch 2/5
40/40 [==============================] - 166s 4s/step - loss: 0.4709 - accuracy: 0.7811 - val_loss: 0.4153 - val_accuracy: 0.8158
Epoch 3/5
40/40 [==============================] - 166s 4s/step - loss: 0.3754 - accuracy: 0.8451 - val_loss: 0.4130 - val_accuracy: 0.8188
Epoch 4/5
40/40 [==============================] - 165s 4s/step - loss: 0.3392 - accuracy: 0.8629 - val_loss: 0.3719 - val_accuracy: 0.8400
Epoch 5/5
40/40 [==============================] - 165s 4s/step - loss: 0.3050 - accuracy: 0.8765 - val_loss: 0.3672 - val_accuracy: 0.8588
Addestramento completato in 831 secondi (5 epoche)
782/782 [==============================] - 85s 109ms/step - loss: 0.3840 - accuracy: 0.8445


[0.38397184014320374, 0.8444799780845642]


Il risultato è livemente inferiore a quello ottenuto con le LSTM ma l'addestramento ha richiesto il 20% del tempo in meno.